# PySpark 数据清洗分析

- 在运行代码之前，你必须确认Pypark的版本和Python的版本是否对应Spark
- 确认Python的环境变量是否修改

In [1]:
#这是一个小测试
from pyspark import SparkConf, SparkContext

conf = SparkConf().setAppName('app').setMaster('local')
sc = SparkContext(conf=conf)
file = 'file:///opt/software/spark/README.md'
data = sc.textFile(file).cache()
numAs = data.filter(lambda line: 'a' in line).count()
numBs = data.filter(lambda line: 'b' in line).count()
print('Line with A: %s , Line with B: %s' % (numAs, numBs))

Line with A: 62 , Line with B: 31


## 清洗股票数据
- 将上证、深证以及创业板的股票个数统计出来

在之前操作中，我们已经将股票列表数据上传至HDFS中，现在我们只需要读取hdfs文件即可进行数据分析

In [3]:
stock_file = 'hdfs:///user/spark/stocks/part-m-00000'
stock_data = sc.textFile(stock_file).cache()
sh_market = stock_data.filter(lambda line: '6' in line[0]).count()
sz_market = stock_data.filter(lambda line: '3' in line[0]).count()
cz_market = stock_data.count() - sh_market - sz_market

数据清洗完成，接下来我们将三大指数的股票写入到本地文件中

In [10]:
f = open('./market_spilt.csv', 'w')
f.writelines('market, numbers \n')
f.writelines('shanghai, {} \n'.format(sh_market))
f.writelines('shenzhen, {} \n'.format(sz_market))
f.writelines('chuangyeban, {} \n'.format(cz_market))
f.close()

我们使用Pandas查看数据的写入情况

In [11]:
import pandas as pd
df = pd.read_csv('./market_spilt.csv')
df

,market,numbers
0,shanghai,2138
1,shenzhen,1089
2,chuangyeban,1392
